# Uso do Aprendizado de Máquina Explicável na Detecção de Discursos de Ódio em Língua Portuguesa

Este notebook implementa um pipeline completo para avaliação e explicabilidade de detectores de discurso de ódio (DO) em **português**, utilizando o corpus **OFFCOMBR** (variantes **OFFCOMBR-2** e **OFFCOMBR-3**). O script realiza carregamento, pré-processamento, treinamento e avaliação de três modelos, produz métricas e gráficos comparativos e gera análises de explicabilidade com SHAP:

1. **Modelo clássico**: Random Forest (TF-IDF + n-grams + features linguísticas).
2. **Modelo transformer**: **BERTimbau** (fine-tuning).
3. **Modelo LLM**: LLaMA3 + LoRA (fine-tuning).

**Estrutura do pipeline**
1. Carregamento dos datasets OFFCOMBR-2 e OFFCOMBR-3 a partir de arquivos ARFF no repositório oficial.
2. Pré-processamento em Português com normalização, remoção de ruído, substituição de gírias, lematização e correção ortográfica opcional.
3. Geração de splits estratificados treino/teste.
4. Treino e avaliação de modelos:
    * Random Forest com pipeline TFIDF ngram + features linguísticas.
    * BERTimbau fine-tuned via Hugging Face Trainer.
    * LLaMA3 fine-tuned com LoRA usando PEFT e treino em 8bit quando disponível.
5. Avaliação e visualização por dataset:
    * Tabela de métricas: Accuracy, Precision, Recall, F1-score.
    * Barplots comparativos.
    * Curvas ROC e Precision Recall.
    * Matrizes de confusão por modelo.
6. Comparação entre OFFCOMBR-2 e OFFCOMBR-3:
    * Tabela consolidada multiindex Dataset × Modelo.
    * Estatísticas de distribuição de classes e proporção de desbalanceamento.
    * Gráficos comparativos de métricas e heatmap de correlação.
7. Explicabilidade com SHAP:
    * TreeExplainer para Random Forest com feature importances (TFIDF + linguísticas).
    * Explainer token-level para BERTimbau e LLaMA3+LoRA; resumo token importances e plots.
    * Saída em CSV e PNG na pasta outputs/shap.

**Arquivos de saída gerados**
* ./results_* diretórios do Trainer para cada dataset e modelo.
* ./outputs/shap/shap_rf_feature_importance.csv PNG e CSV para RF.
* ./outputs/shap/shap_rf_summary.png.
* ./outputs/shap/shap_bert_token_importance.csv.
* ./outputs/shap/shap_bert_summary_bar.png.
* ./outputs/shap/shap_llama_token_importance.csv (se LLaMA disponível).
* ./outputs/shap/shap_llama_summary_bar.png (se LLaMA disponível).
* Plots PNG para curvas ROC, PR, barplots, matrizes de confusão e heatmaps na sessão de execução.
* Console prints contendo tabelas finais e estatísticas de distribuição.

**Dependências e instalação**
* Essenciais:
    * Python 3.10+
    * pip install -U pip setuptools
* Pacotes Python:
    * numpy pandas scikit-learn matplotlib seaborn liac-arff
    * spacy pt_core_news_sm
    * pyspellchecker
    * datasets transformers
    * torch
* Para LLaMA3+LoRA (GPU recomendado):
    * peft bitsandbytes accelerate
    * transformers com suporte a bitsandbytes
    * uma GPU com memória suficiente; recomenda-se CUDA compatível
* Para explicabilidade:
    * shap
    * tqdm
* Exemplo de instalação:
    * pip install numpy pandas scikit-learn matplotlib seaborn liac-arff spacy pyspellchecker datasets transformers torch tqdm shap
    * pip install peft bitsandbytes accelerate # apenas se for usar LoRA
    * python -m spacy download pt_core_news_sm

## Carregamento do Dataset, Pré-processamento e Normalização refinados para português
* Normalização de texto:
    * Conversão para minúsculas.
    * Remoção de acentos (unidecode).
    * Substituição de caracteres especiais.
    * Expansão de contrações (ex.: “vc” → “você”).
* Limpeza:
    * Remoção de URLs, menções (@usuario), hashtags e emojis.
    * Remoção de Stopwords.
* Tokenização:
    * Segmentação em palavras ou subpalavras (BPE, SentencePiece), especialmente útil em português por causa da morfologia rica.
    * Usa spaCy (pt_core_news_sm) para segmentação mais robusta.
* Lematização:
    * Melhor que stemming para preservar sentido.
    * Redução de palavras à forma base (ex.: “xingando” → “xingar”).
* Correção ortográfica e gírias:
    * Biblioteca pyspellchecker para português.
    * Dicionário customizado de gírias → forma padrão (ex.: “vc” → “você”, “pq” → “porque”).

## Random Forest + Feature Engineering Tradicional
* Bag-of-Words (BoW): baseline simples.
* TF-IDF: mais robusto para capturar relevância.
* N-grams: bigramas e trigramas são úteis em discurso de ódio (ex.: “vai morrer”, “seu lixo”):
    * Sugestão: usar 1-3 n-grams.
* Features linguísticas adicionais:
    * Contagem de palavrões (lista customizada).
    * Número de maiúsculas.
    * Número de pontos de exclamação.
    * Emojis detectados.

## BERTimbau (Transformer - variação do BERT)
* Usa transformers (Hugging Face).
* Estratégia: fine-tuning em BERTimbau-base ou BERTimbau-large.
* Entrada: texto cru (sem lematização/stemming, apenas limpeza mínima).
* Saída: classificação binária.
* Word embeddings pré-treinados:
    * Word2Vec, FastText (bom para português por lidar com morfologia e palavras raras).
* Contextual embeddings:
    * BERTimbau e GPT-like models ajustados para português.
* Fine-tuning de Transformers:
    * Ajuste fino em datasets anotados de discurso de ódio.
* Data augmentation:
    * Parafraseamento, tradução ida-e-volta (back-translation), substituição por sinônimos para lidar com desbalanceamento de classes.

## Observações
* Random Forest: baseline clássico, útil para entender ganhos relativos.
* BERTimbau: modelo contextual já pré-treinado em português, se adapta bem ao domínio devido ao ajuste via fine-tuning.
* LLaMA3 + LoRA: não utilizado apenas como um modelo genérico em prompting, foi adaptado ao OFFCOMBR, aprendendo padrões ofensivos específicos do português.
    * Com isso, o LLaMA3 ganha competitividade real contra o BERTimbau.
    * A comparação passa a ser mais justa, porque ambos (BERTimbau e LLaMA3) estão sendo fine-tunados no mesmo dataset.
    * A diferença é que, com LoRA, o LLaMA3 deixa de ser apenas um baseline de prompting e passa a ser um concorrente direto do BERTimbau.
    * Isso enriquece a análise, porque podemos ver quanto o fine-tuning eficiente (LoRA) melhora um LLM genérico em relação a um modelo já especializado em português (BERTimbau).

## Avaliação dos modelos
* Comparação de métricas.
* Curvas ROC e AUC.
* Curvas Precision-Recall (PRC).
* Barplot comparativo.

## Versão CPU + GPU

In [1]:
"""
Pipeline end-to-end único:
1) Carrega OFFCOMBR-2 e OFFCOMBR-3
2) Pré-processa (lemmatização, normalização, correção opcional)
3) Gera splits estratificados
4) Treina/avalia: Random Forest, BERTimbau, LLaMA3+LoRA
5) Plota métricas, ROC, PRC, matrizes de confusão por dataset
6) Consolida comparação entre OFFCOMBR-2 e OFFCOMBR-3 (tabela, distribuição, gráficos, heatmap)
7) Explicabilidade com SHAP (RF, BERTimbau, LLaMA quando disponível)
Notas:
- Ajuste hiperparâmetros, caminhos e batch sizes conforme sua infra.
- Requer pacotes listados no README associado.
"""

# =========================
# 0. Imports gerais e detecção de GPU
# =========================
import os
import urllib.request
import re
import unidecode
import random
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arff
import spacy
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    roc_curve, auc, precision_recall_curve, average_precision_score,
    confusion_matrix
)
from datasets import Dataset
import torch

# Hugging Face / PEFT / SHAP
from transformers import (
    BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments,
    AutoTokenizer, AutoModelForSequenceClassification
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import shap
from tqdm.auto import tqdm

sns.set(style="whitegrid")
plt.rcParams.update({"figure.max_open_warning": 0})

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(f"USE_CUDA={USE_CUDA}, device={device}")

# =========================
# 1. Carregamento OFFCOMBR
# =========================
GITHUB_RAW_BASE = "https://raw.githubusercontent.com/rogersdepelle/OffComBR/master"
ARFF_URLS = {
    "offcombr-2": f"{GITHUB_RAW_BASE}/offcombr2.arff",
    "offcombr-3": f"{GITHUB_RAW_BASE}/offcombr3.arff",
}

def load_offcombr(cfg="offcombr-2"):
    if cfg not in ARFF_URLS:
        raise ValueError("Use 'offcombr-2' ou 'offcombr-3'")
    raw = urllib.request.urlopen(ARFF_URLS[cfg]).read().decode("utf-8", errors="ignore")
    obj = arff.loads(raw)
    cols = [a[0] for a in obj["attributes"]]
    df = pd.DataFrame(obj["data"], columns=cols)
    if "document" in df.columns:
        df = df.rename(columns={"document": "text"})
    df["text"] = df["text"].astype(str).str.strip()
    df = df[df["text"] != ""].copy()
    class_col = next((c for c in df.columns if "class" in c.lower()), None)
    if class_col is None:
        raise ValueError("Não foi encontrada coluna de classe no ARFF")
    df["label_int"] = df[class_col].astype(str).str.lower().map({"no":0,"yes":1,"not":0,"offensive":1}).astype(int)
    return df[["text","label_int"]]

# =========================
# 2. Pré-processamento (Português)
# =========================
nlp = spacy.load("pt_core_news_sm")
spell = SpellChecker(language="pt")
giria_dict = {"vc":"você","pq":"porque","tbm":"também","q":"que"}

def preprocess_text_pt(text, do_spell=True):
    text = str(text).lower()
    text = unidecode.unidecode(text)
    text = re.sub(r"http\S+|@\w+|#\w+|\s+", " ", text)
    for g, norm in giria_dict.items():
        text = re.sub(rf"\b{g}\b", norm, text)
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.is_alpha and not token.is_stop:
            lemma = token.lemma_
            if do_spell:
                corr = spell.correction(token.text)
                tokens.append(corr if corr is not None and corr != token.text else lemma)
            else:
                tokens.append(lemma)
    return " ".join(tokens)

def make_splits(df, test_size=0.2, seed=42):
    X = df["text"].apply(preprocess_text_pt)
    y = df["label_int"]
    X_train, X_test, y_train, y_test = train_test_split(
        X.values, y.values, test_size=test_size, stratify=y.values, random_state=seed
    )
    return X_train, X_test, y_train, y_test

# =========================
# 3. Modelos: Random Forest pipeline
# =========================
class LinguisticFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        feats = []
        for text in X:
            toks = text.split()
            insults = sum(1 for w in toks if w in {"idiota","burro","lixo","estúpido","imbecil"})
            upper_chars = sum(1 for c in text if c.isupper())
            excl = text.count("!")
            feats.append([insults, upper_chars, excl])
        return np.array(feats)

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=10000)
pipeline_rf = Pipeline([
    ("features", FeatureUnion([("tfidf", tfidf), ("ling", LinguisticFeatures())])),
    ("clf", RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))
])

# =========================
# 4. Função genérica de avaliação
# =========================
def avaliar_modelo(nome, modelo, X_test_raw, y_test, tipo="sklearn"):
    if tipo == "sklearn":
        y_pred = modelo.predict(X_test_raw)
        y_score = modelo.predict_proba(X_test_raw)[:,1]
    elif tipo == "trainer":
        preds = modelo.predict(X_test_raw)  # X_test_raw espera HF Dataset (tokenizado)
        logits = preds.predictions
        if logits.ndim == 3:
            logits = logits[:,0,:]
        y_score = logits[:,1]
        y_pred = logits.argmax(axis=-1)
    else:
        raise ValueError("Tipo inválido")
    acc = accuracy_score(y_test, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="weighted", zero_division=0)
    return {"Modelo": nome, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-score": f1, "y_score": y_score}

# =========================
# 5. Função LLaMA3 + LoRA (GPU-aware)
# =========================
def treinar_llama_lora(cfg, X_train, y_train, X_eval, y_eval, output_dir_base="./results_llama"):
    model_name = "meta-llama/Llama-3-8b-instruct"  # substitua conforme o checkpoint do modelo
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    max_len = 256

    ds_train = Dataset.from_dict({"text": X_train.tolist(), "label": y_train})
    ds_eval  = Dataset.from_dict({"text": X_eval.tolist(), "label": y_eval})
    def tokenize_fn(batch): return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=max_len)
    ds_train = ds_train.map(tokenize_fn, batched=True)
    ds_eval  = ds_eval.map(tokenize_fn, batched=True)
    ds_train.set_format("torch", columns=["input_ids","attention_mask","label"])
    ds_eval.set_format("torch", columns=["input_ids","attention_mask","label"])

    # Carrega o modelo em 8-bit e usa device_map para colocar na GPU
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        load_in_8bit=True,
        device_map="auto",
    )

    # Prepara para treinamento k-bit
    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj","v_proj","k_proj","o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_CLS"
    )
    model = get_peft_model(model, lora_config)

    outdir = os.path.join(output_dir_base, cfg, "llama_lora")
    os.makedirs(outdir, exist_ok=True)

    # Se CUDA estiver disponível, habilita fp16; caso contrário roda em fp32
    training_args = TrainingArguments(
        output_dir=outdir,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=3,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        fp16=USE_CUDA,
        logging_steps=50,
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds_train,
        eval_dataset=ds_eval,
        tokenizer=tokenizer,
    )

    trainer.train()
    preds = trainer.predict(ds_eval)
    logits = preds.predictions
    if logits.ndim == 3:
        logits = logits[:,0,:]
    y_score = logits[:,1]
    y_pred = logits.argmax(axis=-1)

    acc = accuracy_score(y_eval, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_eval, y_pred, average="weighted", zero_division=0)
    return {"Modelo": "LLaMA3 + LoRA", "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-score": f1, "y_score": y_score}

# =========================
# 6. Loop principal por dataset: treino e avaliação
# =========================
comparacao_datasets = {}
comparacao_datasets_raw = {}

for cfg in ["offcombr-2", "offcombr-3"]:
    print(f"\n--- Pipeline para {cfg} ---")
    df = load_offcombr(cfg)
    comparacao_datasets_raw[cfg] = df.copy()
    X_train, X_test, y_train, y_test = make_splits(df)

    # Random Forest (CPU)
    print("Treinando Random Forest...")
    pipeline_rf.fit(X_train, y_train)
    res_rf = avaliar_modelo("Random Forest (TF-IDF+n-grams)", pipeline_rf, X_test, y_test, tipo="sklearn")

    # BERTimbau (GPU)
    print("Treinando BERTimbau (uso de GPU quando disponível)...")
    tokenizer_bert = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
    ds_train = Dataset.from_dict({"text": X_train.tolist(), "label": y_train})
    ds_test  = Dataset.from_dict({"text": X_test.tolist(), "label": y_test})
    def tokenize_fn(batch): return tokenizer_bert(batch["text"], padding="max_length", truncation=True, max_length=128)
    ds_train = ds_train.map(tokenize_fn, batched=True)
    ds_test  = ds_test.map(tokenize_fn, batched=True)
    ds_train.set_format("torch", columns=["input_ids","attention_mask","label"])
    ds_test.set_format("torch", columns=["input_ids","attention_mask","label"])

    model_bert = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)
    # Move o modelo para device (GPU se disponível)
    model_bert.to(device)

    # Habilita fp16 se CUDA estiver disponível
    bert_training_args = TrainingArguments(
        output_dir=f"./results_{cfg}/bert",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        logging_steps=50,
        save_strategy="no",
        fp16=USE_CUDA,
    )
    trainer_bert = Trainer(model=model_bert, args=bert_training_args, train_dataset=ds_train, eval_dataset=ds_test, tokenizer=tokenizer_bert)
    trainer_bert.train()
    res_bert = avaliar_modelo("BERTimbau (fine-tune)", trainer_bert, ds_test, y_test, tipo="trainer")

    # LLaMA3 + LoRA (GPU)
    print("Treinando LLaMA3 + LoRA (GPU se disponível)...")
    try:
        res_llama = treinar_llama_lora(cfg, X_train, y_train, X_test, y_test)
        resultados = [res_rf, res_bert, res_llama]
    except Exception as e:
        print("LLaMA3+LoRA falhou ou indisponível:", e)
        resultados = [res_rf, res_bert]

    comparacao_datasets[cfg] = pd.DataFrame([{k:v for k,v in r.items() if k!="y_score"} for r in resultados])
    comparacao_datasets[cfg + "_raw_results"] = resultados
    comparacao_datasets[cfg + "_ytest"] = y_test
    comparacao_datasets[cfg + "_Xtest"] = X_test

# =========================
# 7. Seção de avaliação por dataset (métricas, ROC, PR, conf matrix)
# =========================
def avaliar_resultados(resultados, y_test, cfg_name):
    df_resultados = pd.DataFrame([{k:v for k,v in r.items() if k!="y_score"} for r in resultados])
    print(f"\n=== Comparação de Modelos - {cfg_name.upper()} ===")
    print(df_resultados)

    # Barplot
    melted = df_resultados.melt(id_vars="Modelo", var_name="Métrica", value_name="Valor")
    plt.figure(figsize=(10,6))
    sns.barplot(data=melted[melted["Métrica"]!="Modelo"], x="Métrica", y="Valor", hue="Modelo")
    plt.title(f"Comparação de Modelos - {cfg_name.upper()}")
    plt.ylim(0,1)
    plt.show()

    # ROC
    plt.figure(figsize=(8,6))
    for r in resultados:
        fpr, tpr, _ = roc_curve(y_test, r["y_score"])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f"{r['Modelo']} (AUC = {roc_auc:.2f})")
    plt.plot([0,1],[0,1], color="gray", linestyle="--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(f"Curvas ROC - {cfg_name.upper()}"); plt.legend(loc="lower right"); plt.show()

    # PRC
    plt.figure(figsize=(8,6))
    for r in resultados:
        precision, recall, _ = precision_recall_curve(y_test, r["y_score"])
        ap = average_precision_score(y_test, r["y_score"])
        plt.plot(recall, precision, lw=2, label=f"{r['Modelo']} (AP = {ap:.2f})")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(f"Curvas Precision-Recall - {cfg_name.upper()}"); plt.legend(loc="lower left"); plt.show()

    # Matriz de Confusão
    for r in resultados:
        y_pred = (r["y_score"] >= 0.5).astype(int)
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(5,4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=["Não-ofensivo","Ofensivo"],
                    yticklabels=["Não-ofensivo","Ofensivo"])
        plt.title(f"Matriz de Confusão - {r['Modelo']} ({cfg_name.upper()})")
        plt.ylabel("Verdadeiro"); plt.xlabel("Predito"); plt.show()

    return df_resultados

for cfg in ["offcombr-2", "offcombr-3"]:
    resultados = comparacao_datasets[cfg + "_raw_results"]
    y_test = comparacao_datasets[cfg + "_ytest"]
    _ = avaliar_resultados(resultados, y_test, cfg)

# =========================
# 8. Comparação entre datasets (tabela consolidada, distribuição, gráficos, heatmap)
# =========================
tabela_final = pd.concat(
    {k: v.set_index("Modelo") for k, v in comparacao_datasets.items() if not k.endswith("_raw_results") and not k.endswith("_ytest") and not k.endswith("_Xtest")},
    axis=0
)
print("\n=== Tabela consolidada (OFFCOMBR-2 vs OFFCOMBR-3) ===")
print(tabela_final)

# Estatísticas de distribuição
print("\n=== Distribuição de classes e desbalanceamento ===")
dist_stats = []
for cfg in ["offcombr-2", "offcombr-3"]:
    df_raw = comparacao_datasets_raw[cfg]
    total = len(df_raw)
    ofensivos = int(df_raw["label_int"].sum())
    nao_ofensivos = total - ofensivos
    pct_of = ofensivos / total
    ratio = ofensivos / (nao_ofensivos if nao_ofensivos > 0 else 1)
    dist_stats.append({"Dataset": cfg, "Total": total, "Ofensivo": ofensivos, "Não-ofensivo": nao_ofensivos,
                       "Pct_ofensivo": pct_of, "Pct_nao_ofensivo": 1-pct_of, "Ratio_of/nao": ratio})
    print(f"{cfg.upper()}: total={total} | ofensivo={ofensivos} ({pct_of:.1%}) | não-ofensivo={nao_ofensivos} ({1-pct_of:.1%}) | ratio={ratio:.2f}")

df_dist = pd.DataFrame(dist_stats).set_index("Dataset")

# Gráficos comparativos
plot_df = tabela_final.reset_index().rename(columns={"level_0":"Dataset"}).reset_index(drop=True)

plt.figure(figsize=(12,6))
sns.barplot(data=plot_df, x="Modelo", y="F1-score", hue="Dataset")
plt.title("Comparação de F1-score entre OFFCOMBR-2 e OFFCOMBR-3"); plt.ylim(0,1); plt.show()

plt.figure(figsize=(12,6))
sns.barplot(data=plot_df, x="Modelo", y="Accuracy", hue="Dataset")
plt.title("Comparação de Accuracy entre OFFCOMBR-2 e OFFCOMBR-3"); plt.ylim(0,1); plt.show()

# Heatmap de correlação entre métricas
num_cols = tabela_final.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(8,6))
sns.heatmap(tabela_final[num_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlação entre métricas (OFFCOMBR-2 + OFFCOMBR-3)"); plt.show()

# Distribuição de classes plot
plt.figure(figsize=(8,5))
df_dist_plot = df_dist.reset_index()
sns.barplot(data=df_dist_plot.melt(id_vars="Dataset", value_vars=["Pct_ofensivo","Pct_nao_ofensivo"]),
            x="Dataset", y="value", hue="variable")
plt.ylim(0,1); plt.ylabel("Proporção"); plt.title("Proporção de classes por dataset"); plt.show()

# Mostrar tabelas finais
print("\n--- Tabela final (para leitura) ---")
print(tabela_final.reset_index().rename(columns={"level_0":"Dataset"}))
print("\n--- Distribuição de classes ---")
print(df_dist)

# =========================
# 9. Explicabilidade com SHAP (gera CSVs e PNGs em ./outputs/shap/)
# =========================
os.makedirs("./outputs/shap", exist_ok=True)

# Parâmetros ajustáveis (reduzir para acelerar)
n_background = 50    # tamanho da amostra de fundo para explainers de kernel
n_explain = 100      # quantos exemplos do test set explicar (subamostrar)
n_samples_kernel = 1024  # nsamples para KernelExplainer (quanto maior, mais estável/mais lento)

# Helper: obter textos de teste (subsample)
def sample_test_texts(X_test, n):
    n = min(n, len(X_test))
    idx = np.random.default_rng(42).choice(len(X_test), size=n, replace=False)
    return X_test[idx], idx

# -------------------------
# 9.1. Random Forest (TreeExplainer)
# -------------------------
try:
    print("\n=== SHAP: Random Forest ===")
    # Para TreeExplainer precisamos do transform interno (TF-IDF) + features numéricas.
    # Criado uma wrapper que recebe raw text array e retorna predict_proba.
    def rf_proba(texts):
        return pipeline_rf.predict_proba(texts)

    # background: amostra de treino (pré-processado) — usar X_train já disponível no loop; escolhido do último dataset ou concatenado
    # Escolhido X_train do offcombr-2 se disponível, senão do primeiro dataset
    any_cfg = "offcombr-2" if "offcombr-2" in comparacao_datasets else list(comparacao_datasets.keys())[0]
    X_train_ref = comparacao_datasets[any_cfg + "_Xtest"] if any_cfg + "_Xtest" in comparacao_datasets else X_train
    background_texts = np.random.choice(X_train_ref, size=min(n_background, len(X_train_ref)), replace=False)

    # Aplica masker/text se quiser explicações no nível de tokens não faz sentido para TF-IDF features;
    # usado explainer de árvore direto sobre pipeline (shap.TreeExplainer aceita sklearn pipelines)
    explainer_rf = shap.TreeExplainer(pipeline_rf.named_steps["clf"])  # TreeExplainer sobre o estimador
    # Para obter valores SHAP para as features TF-IDF + linguísticas precisamos extrair matriz de features:
    X_test_full_feats = pipeline_rf.named_steps["features"].transform(comparacao_datasets[any_cfg + "_Xtest"])
    # shap expects 2d array
    shap_values_rf = explainer_rf.shap_values(X_test_full_feats, check_additivity=False)

    # shap_values_rf é uma lista (um por classe) — usamos classe 1 (ofensivo)
    sv_rf_pos = shap_values_rf[1] if isinstance(shap_values_rf, list) else shap_values_rf
    # Agrega importâncias (valor absoluto médio por feature)
    mean_abs = np.abs(sv_rf_pos).mean(axis=0)
    # Reconstrói nomes de features: TF-IDF feature names + linguistic feature names
    tfidf_names = pipeline_rf.named_steps["features"].transformer_list[0][1].get_feature_names_out()
    ling_names = ["insults_count","upper_chars","exclamations"]
    feat_names = list(tfidf_names) + ling_names
    df_shap_rf = pd.DataFrame({"feature": feat_names, "mean_abs_shap": mean_abs})
    df_shap_rf = df_shap_rf.sort_values("mean_abs_shap", ascending=False).reset_index(drop=True)
    df_shap_rf.to_csv("./outputs/shap/shap_rf_feature_importance.csv", index=False)
    print("Saved ./outputs/shap/shap_rf_feature_importance.csv")

    # Summary plot (top 30 features)
    top_k = min(30, len(feat_names))
    plt.figure(figsize=(8,10))
    shap.summary_plot(sv_rf_pos, X_test_full_feats, feature_names=feat_names, max_display=top_k, show=False)
    plt.tight_layout()
    plt.savefig("./outputs/shap/shap_rf_summary.png", dpi=150)
    plt.close()
    print("Saved ./outputs/shap/shap_rf_summary.png")
except Exception as e:
    print("SHAP Random Forest falhou:", e)

# -------------------------
# 9.2. BERTimbau (Transformer) — KernelExplainer com wrapper sobre Trainer.predict ou model pipeline
# -------------------------
try:
    print("\n=== SHAP: BERTimbau ===")
    # Localiza trainer/BERT treinado: usado trainer_bert do último loop (treinado por cfg = offcombr-3 por fim)
    # Treinado para ambos datasets, pego objeto trainer_bert do último loop; definido wrapper predict_proba_texts
    def bert_predict_proba(texts):
        # texts: list/np.array de raw strings
        enc = tokenizer_bert(list(texts), padding=True, truncation=True, max_length=128, return_tensors="pt")
        # Move tensores para device
        enc = {k: v.to(device) for k, v in enc.items()}
        model_bert.to(device)
        with torch.no_grad():
            out = model_bert(**enc)
            probs = torch.softmax(out.logits, dim=-1)[:,1].detach().cpu().numpy()
        # KernelExplainer espera shape (n_samples, ) or (n_samples, n_outputs)
        return np.vstack([1-probs, probs]).T

    # Prepara background (pequena amostra de textos)
    # Usa comparacao_datasets_raw do último dataset (offcombr-3 se existir)
    bg_cfg = "offcombr-3" if "offcombr-3" in comparacao_datasets_raw else list(comparacao_datasets_raw.keys())[0]
    bg_texts = np.random.choice(comparacao_datasets_raw[bg_cfg]["text"].values, size=min(n_background, len(comparacao_datasets_raw[bg_cfg])), replace=False)

    # Seleciona textos para explicar (do test set)
    test_texts, test_idx = sample_test_texts(comparacao_datasets[bg_cfg + "_Xtest"] if bg_cfg + "_Xtest" in comparacao_datasets else X_test, n_explain)

    # KernelExplainer wrapper: usa função que retorna probas para classe 1; shap.KernelExplainer aceita função retornando uma escalar ou vetor
    # Criar função que aceita a list de strings e retorna probas positivas
    def bert_fn(texts):
        return bert_predict_proba(texts)

    # Usa text masker para acelerar e permitir token masking; KernelExplainer ignora masker mas shap.Explainer com masker.Text pode auto-selecionar explainer
    # Usa shap.Explainer (auto) com masker text:
    masker = shap.maskers.Text(tokenizer_bert)
    explainer_bert = shap.Explainer(lambda x: bert_predict_proba(x), masker, output_names=["non-off","offensive"])
    # Explica (pode ser demorado)
    shap_values_bert = explainer_bert(test_texts)

    # Agrega importâncias token-level: para cada sample, shap_values_bert.data (tokens) e shap_values_bert.values (len toks x outputs)
    # Vamos gerar uma tabela que resume importância média absoluta por token text através de explained samples
    tokens_list = []
    for i in range(len(test_texts)):
        toks = shap_values_bert.data[i]
        vals = shap_values_bert.values[i][:,1]  # classe positiva
        for t, v in zip(toks, vals):
            tokens_list.append({"text": test_texts[i], "token": t, "shap_abs": abs(v), "shap": v})
    df_tokens = pd.DataFrame(tokens_list)
    df_token_summary = df_tokens.groupby("token")["shap_abs"].mean().sort_values(ascending=False).reset_index().rename(columns={"shap_abs":"mean_abs_shap"})
    df_token_summary.to_csv("./outputs/shap/shap_bert_token_importance.csv", index=False)
    print("Saved ./outputs/shap/shap_bert_token_importance.csv")

    # Summary plot (per-sample token-level summary)
    plt.figure(figsize=(10,6))
    shap.plots.bar(shap_values_bert[:, :, 1], max_display=30, show=False)  # class index 1
    plt.title("BERTimbau token-level SHAP (class=offensive)")
    plt.savefig("./outputs/shap/shap_bert_summary_bar.png", dpi=150)
    plt.close()
    print("Saved ./outputs/shap/shap_bert_summary_bar.png")
except Exception as e:
    print("SHAP BERTimbau falhou:", e)

# -------------------------
# 9.3. LLaMA3 + LoRA (Transformer via KernelExplainer wrapper)
# -------------------------
try:
    # Verficase existem resultados do LLaMA
    has_llama = any("LLaMA3 + LoRA" in r["Modelo"] for k in comparacao_datasets.keys() for r in (comparacao_datasets.get(k + "_raw_results", []) if isinstance(comparacao_datasets.get(k + "_raw_results", []), list) else []))
    if has_llama:
        print("\n=== SHAP: LLaMA3 + LoRA ===")
        # Procura qual cfg tem os resultados do LLaMA
        llama_cfg = None
        for cfg in ["offcombr-2","offcombr-3"]:
            raw = comparacao_datasets.get(cfg + "_raw_results", [])
            if any(r["Modelo"] == "LLaMA3 + LoRA" for r in raw):
                llama_cfg = cfg
                break
        if llama_cfg is None:
            raise RuntimeError("Não foi encontrado LLaMA3+LoRA nos resultados")

        # Build wrapper: usa trainer retornado de treinar_llama_lora ou usa trainer salvo, caso salvo.
        # Rebuild da função de predição usando AutoTokenizer + get_peft_model modelo carregado, caso disponível.
        # Approach simples: reusa as predições que já foram computadas (y_score e logits) — mas SHAP precisa de acesso ao modelo.
        # Criado um wrapper que carrega o modelo LLaMA (mode de inferência) e retorna as probabilidades para a lista de textos.
        model_name = "meta-llama/Llama-3-8b-instruct"  # mesmo do trainamento
        tokenizer_llama = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        # Carrega o modelo em 8bit + device_map auto (para inferência)
        llama_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, load_in_8bit=True, device_map="auto")
        # Se PEFT weights salvo, carregue peft weights: get_peft_model + model.load_state_dict ou use PeftModel.from_pretrained

        def llama_predict_proba(texts):
            enc = tokenizer_llama(list(texts), padding=True, truncation=True, max_length=256, return_tensors="pt")
            enc = {k: v.to(device) for k, v in enc.items()}
            with torch.no_grad():
                out = llama_model(**enc)
                probs = torch.softmax(out.logits, dim=-1)[:,1].detach().cpu().numpy()
            return np.vstack([1-probs, probs]).T

        # Background e textos de teste
        bg_cfg = llama_cfg
        bg_texts = np.random.choice(comparacao_datasets_raw[bg_cfg]["text"].values, size=min(n_background, len(comparacao_datasets_raw[bg_cfg])), replace=False)
        test_texts, test_idx = sample_test_texts(comparacao_datasets[bg_cfg + "_Xtest"], n_explain)

        masker = shap.maskers.Text(tokenizer_llama)
        explainer_llama = shap.Explainer(lambda x: llama_predict_proba(x), masker, output_names=["non-off","offensive"])
        shap_values_llama = explainer_llama(test_texts)

        # Importância de nível de tokens agregada
        tokens_list = []
        for i in range(len(test_texts)):
            toks = shap_values_llama.data[i]
            vals = shap_values_llama.values[i][:,1]
            for t, v in zip(toks, vals):
                tokens_list.append({"text": test_texts[i], "token": t, "shap_abs": abs(v), "shap": v})
        df_tokens_llama = pd.DataFrame(tokens_list)
        df_token_summary_llama = df_tokens_llama.groupby("token")["shap_abs"].mean().sort_values(ascending=False).reset_index().rename(columns={"shap_abs":"mean_abs_shap"})
        df_token_summary_llama.to_csv("./outputs/shap/shap_llama_token_importance.csv", index=False)
        print("Saved ./outputs/shap/shap_llama_token_importance.csv")

        plt.figure(figsize=(10,6))
        shap.plots.bar(shap_values_llama[:, :, 1], max_display=30, show=False)
        plt.title("LLaMA3+LoRA token-level SHAP (class=offensive)")
        plt.savefig("./outputs/shap/shap_llama_summary_bar.png", dpi=150)
        plt.close()
        print("Saved ./outputs/shap/shap_llama_summary_bar.png")
    else:
        print("Nenhum resultado LLaMA3+LoRA detectado; pulando SHAP para LLaMA.")
except Exception as e:
    print("SHAP LLaMA3+LoRA falhou:", e)

# -------------------------
# 9.4. Consolidação: tabelas resumidas por modelo
# -------------------------
try:
    print("\n=== Consolidação de tabelas SHAP ===")
    # Tabela RF já foi salva; Tabela de tokens BERT/LLaMA salva.
    # Carrega e mostra os top-20 de cada
    if os.path.exists("./outputs/shap/shap_rf_feature_importance.csv"):
        df_rf_shap = pd.read_csv("./outputs/shap/shap_rf_feature_importance.csv").head(20)
        print("\nTop RF features (SHAP):")
        print(df_rf_shap)

    if os.path.exists("./outputs/shap/shap_bert_token_importance.csv"):
        df_bert_tok = pd.read_csv("./outputs/shap/shap_bert_token_importance.csv").head(30)
        print("\nTop BERT tokens (SHAP):")
        print(df_bert_tok)

    if os.path.exists("./outputs/shap/shap_llama_token_importance.csv"):
        df_llama_tok = pd.read_csv("./outputs/shap/shap_llama_token_importance.csv").head(30)
        print("\nTop LLaMA tokens (SHAP):")
        print(df_llama_tok)
except Exception as e:
    print("Consolidação SHAP falhou:", e)

print("\nSHAP analysis complete. Outputs saved to ./outputs/shap/")

# =========================
# Fim do script
# =========================

ModuleNotFoundError: No module named 'arff'

In [1]:
"""
Pipeline end-to-end único:
1) Carrega OFFCOMBR-2 e OFFCOMBR-3
2) Pré-processa (lemmatização, normalização, correção opcional)
3) Gera splits estratificados
4) Treina/avalia: Random Forest, BERTimbau, LLaMA3+LoRA
5) Plota métricas, ROC, PRC, matrizes de confusão por dataset
6) Consolida comparação entre OFFCOMBR-2 e OFFCOMBR-3 (tabela, distribuição, gráficos, heatmap)
7) Explicabilidade com SHAP (RF, BERTimbau, LLaMA quando disponível)
Notas:
- Ajuste hiperparâmetros, caminhos e batch sizes conforme sua infra.
- Requer pacotes listados no README associado.
"""

# =========================
# 0. Imports gerais e detecção de GPU
# =========================
import os
import urllib.request
import re
import unidecode
import random
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arff
import spacy
from spellchecker import SpellChecker
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    roc_curve, auc, precision_recall_curve, average_precision_score,
    confusion_matrix
)
from datasets import Dataset
import torch

# Hugging Face / PEFT / SHAP
from transformers import (
    BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments,
    AutoTokenizer, AutoModelForSequenceClassification
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import shap
from tqdm.auto import tqdm

sns.set(style="whitegrid")
plt.rcParams.update({"figure.max_open_warning": 0})

USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print(f"USE_CUDA={USE_CUDA}, device={device}")

# =========================
# 1. Carregamento OFFCOMBR
# =========================
GITHUB_RAW_BASE = "https://raw.githubusercontent.com/rogersdepelle/OffComBR/master"
ARFF_URLS = {
    "offcombr-2": f"{GITHUB_RAW_BASE}/OffComBR2.arff",
    "offcombr-3": f"{GITHUB_RAW_BASE}/OffComBR3.arff",
}

def load_offcombr(cfg="offcombr-2"):
    if cfg not in ARFF_URLS:
        raise ValueError("Use 'offcombr-2' ou 'offcombr-3'")
    raw = urllib.request.urlopen(ARFF_URLS[cfg]).read().decode("utf-8", errors="ignore")
    obj = arff.loads(raw)
    cols = [a[0] for a in obj["attributes"]]
    df = pd.DataFrame(obj["data"], columns=cols)
    if "document" in df.columns:
        df = df.rename(columns={"document": "text"})
    df["text"] = df["text"].astype(str).str.strip()
    df = df[df["text"] != ""].copy()
    class_col = next((c for c in df.columns if "class" in c.lower()), None)
    if class_col is None:
        raise ValueError("Não foi encontrada coluna de classe no ARFF")
    df["label_int"] = df[class_col].astype(str).str.lower().map({"no":0,"yes":1,"not":0,"offensive":1}).astype(int)
    return df[["text","label_int"]]

# =========================
# 2. Pré-processamento (Português)
# =========================
nlp = spacy.load("pt_core_news_sm")
spell = SpellChecker(language="pt")
giria_dict = {"vc":"você","pq":"porque","tbm":"também","q":"que"}

def preprocess_text_pt(text, do_spell=True):
    text = str(text).lower()
    text = unidecode.unidecode(text)
    text = re.sub(r"http\S+|@\w+|#\w+|\s+", " ", text)
    for g, norm in giria_dict.items():
        text = re.sub(rf"\b{g}\b", norm, text)
    doc = nlp(text)
    tokens = []
    for token in doc:
        if token.is_alpha and not token.is_stop:
            lemma = token.lemma_
            if do_spell:
                corr = spell.correction(token.text)
                tokens.append(corr if corr is not None and corr != token.text else lemma)
            else:
                tokens.append(lemma)
    return " ".join(tokens)

def make_splits(df, test_size=0.2, seed=42):
    X = df["text"].apply(preprocess_text_pt)
    y = df["label_int"]
    X_train, X_test, y_train, y_test = train_test_split(
        X.values, y.values, test_size=test_size, stratify=y.values, random_state=seed
    )
    return X_train, X_test, y_train, y_test

# =========================
# 3. Modelos: Random Forest pipeline
# =========================
class LinguisticFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        feats = []
        for text in X:
            toks = text.split()
            insults = sum(1 for w in toks if w in {"idiota","burro","lixo","estúpido","imbecil"})
            upper_chars = sum(1 for c in text if c.isupper())
            excl = text.count("!")
            feats.append([insults, upper_chars, excl])
        return np.array(feats)

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=10000)
pipeline_rf = Pipeline([
    ("features", FeatureUnion([("tfidf", tfidf), ("ling", LinguisticFeatures())])),
    ("clf", RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1))
])

# =========================
# 4. Função genérica de avaliação
# =========================
def avaliar_modelo(nome, modelo, X_test_raw, y_test, tipo="sklearn"):
    if tipo == "sklearn":
        y_pred = modelo.predict(X_test_raw)
        y_score = modelo.predict_proba(X_test_raw)[:,1]
    elif tipo == "trainer":
        preds = modelo.predict(X_test_raw)
        logits = preds.predictions
        if logits.ndim == 3:
            logits = logits[:,0,:]
        y_score = logits[:,1]
        y_pred = logits.argmax(axis=-1)
    else:
        raise ValueError("Tipo inválido")
    acc = accuracy_score(y_test, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average="weighted", zero_division=0)
    return {"Modelo": nome, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-score": f1, "y_score": y_score}

# =========================
# 5. Função LLaMA3 + LoRA (integração GPU-aware)
# =========================
def treinar_llama_lora(cfg, X_train, y_train, X_eval, y_eval, output_dir_base="./results_llama"):
    model_name = "meta-llama/Llama-3-8b-instruct"  # substituir conforme checkpoint disponível
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    max_len = 256

    ds_train = Dataset.from_dict({"text": X_train.tolist(), "label": y_train})
    ds_eval  = Dataset.from_dict({"text": X_eval.tolist(), "label": y_eval})
    def tokenize_fn(batch): return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=max_len)
    ds_train = ds_train.map(tokenize_fn, batched=True)
    ds_eval  = ds_eval.map(tokenize_fn, batched=True)
    ds_train.set_format("torch", columns=["input_ids","attention_mask","label"])
    ds_eval.set_format("torch", columns=["input_ids","attention_mask","label"])

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=2,
        load_in_8bit=True,
        device_map="auto",
    )

    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj","v_proj","k_proj","o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="SEQ_CLS"
    )
    model = get_peft_model(model, lora_config)

    outdir = os.path.join(output_dir_base, cfg, "llama_lora")
    os.makedirs(outdir, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=outdir,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=3,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        fp16=USE_CUDA,
        logging_steps=50,
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds_train,
        eval_dataset=ds_eval,
        tokenizer=tokenizer,
    )

    trainer.train()
    preds = trainer.predict(ds_eval)
    logits = preds.predictions
    if logits.ndim == 3:
        logits = logits[:,0,:]
    y_score = logits[:,1]
    y_pred = logits.argmax(axis=-1)

    acc = accuracy_score(y_eval, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_eval, y_pred, average="weighted", zero_division=0)
    return {"Modelo": "LLaMA3 + LoRA", "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-score": f1, "y_score": y_score}

# =========================
# 6. Loop principal por dataset: treino e avaliação
# =========================
comparacao_datasets = {}
comparacao_datasets_raw = {}

for cfg in ["offcombr-2", "offcombr-3"]:
    print(f"\n--- Pipeline para {cfg} ---")
    df = load_offcombr(cfg)
    comparacao_datasets_raw[cfg] = df.copy()
    X_train, X_test, y_train, y_test = make_splits(df)

    # Random Forest
    print("Treinando Random Forest...")
    pipeline_rf.fit(X_train, y_train)
    res_rf = avaliar_modelo("Random Forest (TF-IDF+n-grams)", pipeline_rf, X_test, y_test, tipo="sklearn")

    # BERTimbau
    print("Treinando BERTimbau (uso de GPU se disponível)...")
    tokenizer_bert = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
    ds_train = Dataset.from_dict({"text": X_train.tolist(), "label": y_train})
    ds_test  = Dataset.from_dict({"text": X_test.tolist(), "label": y_test})
    def tokenize_fn(batch): return tokenizer_bert(batch["text"], padding="max_length", truncation=True, max_length=128)
    ds_train = ds_train.map(tokenize_fn, batched=True)
    ds_test  = ds_test.map(tokenize_fn, batched=True)
    ds_train.set_format("torch", columns=["input_ids","attention_mask","label"])
    ds_test.set_format("torch", columns=["input_ids","attention_mask","label"])
    model_bert = BertForSequenceClassification.from_pretrained("neuralmind/bert-base-portuguese-cased", num_labels=2)
    model_bert.to(device)
    bert_args = TrainingArguments(
        output_dir=f"./results_{cfg}/bert",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        logging_steps=50,
        save_strategy="no",
        fp16=USE_CUDA,
    )
    trainer_bert = Trainer(model=model_bert, args=bert_args, train_dataset=ds_train, eval_dataset=ds_test, tokenizer=tokenizer_bert)
    trainer_bert.train()
    res_bert = avaliar_modelo("BERTimbau (fine-tune)", trainer_bert, ds_test, y_test, tipo="trainer")

    # LLaMA3 + LoRA
    print("Treinando LLaMA3 + LoRA (GPU se disponível; pode falhar sem infra apropriada)...")
    try:
        res_llama = treinar_llama_lora(cfg, X_train, y_train, X_test, y_test)
        resultados = [res_rf, res_bert, res_llama]
    except Exception as e:
        print("LLaMA3+LoRA falhou ou indisponível:", e)
        resultados = [res_rf, res_bert]

    comparacao_datasets[cfg] = pd.DataFrame([{k:v for k,v in r.items() if k!="y_score"} for r in resultados])
    comparacao_datasets[cfg + "_raw_results"] = resultados
    comparacao_datasets[cfg + "_ytest"] = y_test
    comparacao_datasets[cfg + "_Xtest"] = X_test

# =========================
# 7. Seção de avaliação por dataset (métricas, ROC, PR, conf matrix)
# =========================
def avaliar_resultados(resultados, y_test, cfg_name):
    df_resultados = pd.DataFrame([{k:v for k,v in r.items() if k!="y_score"} for r in resultados])
    print(f"\n=== Comparação de Modelos - {cfg_name.upper()} ===")
    print(df_resultados)

    melted = df_resultados.melt(id_vars="Modelo", var_name="Métrica", value_name="Valor")
    plt.figure(figsize=(10,6))
    sns.barplot(data=melted[melted["Métrica"]!="Modelo"], x="Métrica", y="Valor", hue="Modelo")
    plt.title(f"Comparação de Modelos - {cfg_name.upper()}"); plt.ylim(0,1); plt.show()

    plt.figure(figsize=(8,6))
    for r in resultados:
        fpr, tpr, _ = roc_curve(y_test, r["y_score"])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f"{r['Modelo']} (AUC = {roc_auc:.2f})")
    plt.plot([0,1],[0,1], color="gray", linestyle="--")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(f"Curvas ROC - {cfg_name.upper()}"); plt.legend(loc="lower right"); plt.show()

    plt.figure(figsize=(8,6))
    for r in resultados:
        precision, recall, _ = precision_recall_curve(y_test, r["y_score"])
        ap = average_precision_score(y_test, r["y_score"])
        plt.plot(recall, precision, lw=2, label=f"{r['Modelo']} (AP = {ap:.2f})")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(f"Curvas Precision-Recall - {cfg_name.upper()}"); plt.legend(loc="lower left"); plt.show()

    for r in resultados:
        y_pred = (r["y_score"] >= 0.5).astype(int)
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(5,4))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                    xticklabels=["Não-ofensivo","Ofensivo"],
                    yticklabels=["Não-ofensivo","Ofensivo"])
        plt.title(f"Matriz de Confusão - {r['Modelo']} ({cfg_name.upper()})")
        plt.ylabel("Verdadeiro"); plt.xlabel("Predito"); plt.show()

    return df_resultados

for cfg in ["offcombr-2", "offcombr-3"]:
    resultados = comparacao_datasets[cfg + "_raw_results"]
    y_test = comparacao_datasets[cfg + "_ytest"]
    _ = avaliar_resultados(resultados, y_test, cfg)

# =========================
# 8. Comparação entre datasets (tabela consolidada, distribuição, gráficos, heatmap)
# =========================
tabela_final = pd.concat(
    {k: v.set_index("Modelo") for k, v in comparacao_datasets.items() if not k.endswith("_raw_results") and not k.endswith("_ytest") and not k.endswith("_Xtest")},
    axis=0
)
print("\n=== Tabela consolidada (OFFCOMBR-2 vs OFFCOMBR-3) ===")
print(tabela_final)

dist_stats = []
for cfg in ["offcombr-2", "offcombr-3"]:
    df_raw = comparacao_datasets_raw[cfg]
    total = len(df_raw)
    ofensivos = int(df_raw["label_int"].sum())
    nao_ofensivos = total - ofensivos
    pct_of = ofensivos / total
    ratio = ofensivos / (nao_ofensivos if nao_ofensivos > 0 else 1)
    dist_stats.append({"Dataset": cfg, "Total": total, "Ofensivo": ofensivos, "Não-ofensivo": nao_ofensivos,
                       "Pct_ofensivo": pct_of, "Pct_nao_ofensivo": 1-pct_of, "Ratio_of/nao": ratio})
    print(f"{cfg.upper()}: total={total} | ofensivo={ofensivos} ({pct_of:.1%}) | não-ofensivo={nao_ofensivos} ({1-pct_of:.1%}) | ratio={ratio:.2f}")

df_dist = pd.DataFrame(dist_stats).set_index("Dataset")

plot_df = tabela_final.reset_index().rename(columns={"level_0":"Dataset"}).reset_index(drop=True)
plt.figure(figsize=(12,6))
sns.barplot(data=plot_df, x="Modelo", y="F1-score", hue="Dataset"); plt.title("Comparação de F1-score entre OFFCOMBR-2 e OFFCOMBR-3"); plt.ylim(0,1); plt.show()
plt.figure(figsize=(12,6))
sns.barplot(data=plot_df, x="Modelo", y="Accuracy", hue="Dataset"); plt.title("Comparação de Accuracy entre OFFCOMBR-2 e OFFCOMBR-3"); plt.ylim(0,1); plt.show()

num_cols = tabela_final.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(8,6))
sns.heatmap(tabela_final[num_cols].corr(), annot=True, cmap="coolwarm", fmt=".2f"); plt.title("Correlação entre métricas (OFFCOMBR-2 + OFFCOMBR-3)"); plt.show()

plt.figure(figsize=(8,5))
df_dist_plot = df_dist.reset_index()
sns.barplot(data=df_dist_plot.melt(id_vars="Dataset", value_vars=["Pct_ofensivo","Pct_nao_ofensivo"]),
            x="Dataset", y="value", hue="variable"); plt.ylim(0,1); plt.ylabel("Proporção"); plt.title("Proporção de classes por dataset"); plt.show()

print("\n--- Tabela final (para leitura) ---")
print(tabela_final.reset_index().rename(columns={"level_0":"Dataset"}))
print("\n--- Distribuição de classes ---")
print(df_dist)

# =========================
# 9. Explicabilidade com SHAP (gera CSVs e PNGs em ./outputs/shap/)
# =========================
os.makedirs("./outputs/shap", exist_ok=True)
n_background = 50
n_explain = 100

def sample_test_texts(X_test, n):
    n = min(n, len(X_test))
    idx = np.random.default_rng(42).choice(len(X_test), size=n, replace=False)
    return X_test[idx], idx

# 9.A Random Forest (TreeExplainer)
try:
    print("\n=== SHAP: Random Forest ===")
    any_cfg = "offcombr-2"
    if any_cfg + "_Xtest" in comparacao_datasets:
        test_texts = comparacao_datasets[any_cfg + "_Xtest"]
    else:
        test_texts = X_test
    X_test_full_feats = pipeline_rf.named_steps["features"].transform(test_texts)
    explainer_rf = shap.TreeExplainer(pipeline_rf.named_steps["clf"])
    shap_values_rf = explainer_rf.shap_values(X_test_full_feats, check_additivity=False)
    sv_rf_pos = shap_values_rf[1] if isinstance(shap_values_rf, list) else shap_values_rf
    mean_abs = np.abs(sv_rf_pos).mean(axis=0)
    tfidf_names = pipeline_rf.named_steps["features"].transformer_list[0][1].get_feature_names_out()
    ling_names = ["insults_count","upper_chars","exclamations"]
    feat_names = list(tfidf_names) + ling_names
    df_shap_rf = pd.DataFrame({"feature": feat_names, "mean_abs_shap": mean_abs}).sort_values("mean_abs_shap", ascending=False)
    df_shap_rf.to_csv("./outputs/shap/shap_rf_feature_importance.csv", index=False)
    plt.figure(figsize=(8,10))
    shap.summary_plot(sv_rf_pos, X_test_full_feats, feature_names=feat_names, max_display=30, show=False)
    plt.tight_layout(); plt.savefig("./outputs/shap/shap_rf_summary.png", dpi=150); plt.close()
    print("Saved ./outputs/shap/shap_rf_feature_importance.csv and shap_rf_summary.png")
except Exception as e:
    print("SHAP Random Forest falhou:", e)

# 9.B BERTimbau token-level (shap.Explainer with text masker)
try:
    print("\n=== SHAP: BERTimbau ===")
    # Use model_bert and tokenizer_bert from last loop (they exist in namespace)
    bg_cfg = "offcombr-3" if "offcombr-3" in comparacao_datasets_raw else "offcombr-2"
    bg_texts = np.random.choice(comparacao_datasets_raw[bg_cfg]["text"].values, size=min(n_background, len(comparacao_datasets_raw[bg_cfg])), replace=False)
    test_texts, _ = sample_test_texts(comparacao_datasets[bg_cfg + "_Xtest"] if bg_cfg + "_Xtest" in comparacao_datasets else X_test, n_explain)
    tokenizer_local = tokenizer_bert
    model_local = model_bert
    def bert_predict_proba(texts):
        enc = tokenizer_local(list(texts), padding=True, truncation=True, max_length=128, return_tensors="pt")
        enc = {k: v.to(device) for k, v in enc.items()}
        model_local.to(device)
        with torch.no_grad():
            out = model_local(**enc)
            probs = torch.softmax(out.logits, dim=-1)[:,1].detach().cpu().numpy()
        return np.vstack([1-probs, probs]).T
    masker = shap.maskers.Text(tokenizer_local)
    explainer_bert = shap.Explainer(lambda x: bert_predict_proba(x), masker, output_names=["non-off","offensive"])
    shap_values_bert = explainer_bert(test_texts)
    tokens_list = []
    for i in range(len(test_texts)):
        toks = shap_values_bert.data[i]
        vals = shap_values_bert.values[i][:,1]
        for t, v in zip(toks, vals):
            tokens_list.append({"text": test_texts[i], "token": t, "shap_abs": abs(v), "shap": v})
    df_tokens = pd.DataFrame(tokens_list)
    df_token_summary = df_tokens.groupby("token")["shap_abs"].mean().sort_values(ascending=False).reset_index().rename(columns={"shap_abs":"mean_abs_shap"})
    df_token_summary.to_csv("./outputs/shap/shap_bert_token_importance.csv", index=False)
    plt.figure(figsize=(10,6))
    shap.plots.bar(shap_values_bert[:,:,1], max_display=30, show=False)
    plt.title("BERTimbau token-level SHAP (class=offensive)")
    plt.savefig("./outputs/shap/shap_bert_summary_bar.png", dpi=150); plt.close()
    print("Saved ./outputs/shap/shap_bert_token_importance.csv and shap_bert_summary_bar.png")
except Exception as e:
    print("SHAP BERTimbau falhou:", e)

# 9.C LLaMA3 + LoRA token-level (when available)
try:
    has_llama = any(any(r.get("Modelo","")=="LLaMA3 + LoRA" for r in comparacao_datasets.get(cfg + "_raw_results", [])) for cfg in ["offcombr-2","offcombr-3"])
    if has_llama:
        print("\n=== SHAP: LLaMA3 + LoRA ===")
        llama_cfg = None
        for cfg in ["offcombr-2","offcombr-3"]:
            raw = comparacao_datasets.get(cfg + "_raw_results", [])
            if any(r.get("Modelo","")== "LLaMA3 + LoRA" for r in raw):
                llama_cfg = cfg
                break
        model_name = "meta-llama/Llama-3-8b-instruct"
        tokenizer_llama = AutoTokenizer.from_pretrained(model_name, use_fast=True)
        llama_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, load_in_8bit=True, device_map="auto")
        def llama_predict_proba(texts):
            enc = tokenizer_llama(list(texts), padding=True, truncation=True, max_length=256, return_tensors="pt")
            enc = {k: v.to(device) for k, v in enc.items()}
            with torch.no_grad():
                out = llama_model(**enc)
                probs = torch.softmax(out.logits, dim=-1)[:,1].detach().cpu().numpy()
            return np.vstack([1-probs, probs]).T
        test_texts, _ = sample_test_texts(comparacao_datasets[llama_cfg + "_Xtest"], n_explain)
        masker = shap.maskers.Text(tokenizer_llama)
        explainer_llama = shap.Explainer(lambda x: llama_predict_proba(x), masker, output_names=["non-off","offensive"])
        shap_values_llama = explainer_llama(test_texts)
        tokens_list = []
        for i in range(len(test_texts)):
            toks = shap_values_llama.data[i]
            vals = shap_values_llama.values[i][:,1]
            for t, v in zip(toks, vals):
                tokens_list.append({"text": test_texts[i], "token": t, "shap_abs": abs(v), "shap": v})
        df_tokens_llama = pd.DataFrame(tokens_list)
        df_token_summary_llama = df_tokens_llama.groupby("token")["shap_abs"].mean().sort_values(ascending=False).reset_index().rename(columns={"shap_abs":"mean_abs_shap"})
        df_token_summary_llama.to_csv("./outputs/shap/shap_llama_token_importance.csv", index=False)
        plt.figure(figsize=(10,6))
        shap.plots.bar(shap_values_llama[:,:,1], max_display=30, show=False)
        plt.title("LLaMA3+LoRA token-level SHAP (class=offensive)")
        plt.savefig("./outputs/shap/shap_llama_summary_bar.png", dpi=150); plt.close()
        print("Saved ./outputs/shap/shap_llama_token_importance.csv and shap_llama_summary_bar.png")
    else:
        print("Nenhum resultado LLaMA3+LoRA detectado; pulando SHAP para LLaMA.")
except Exception as e:
    print("SHAP LLaMA3+LoRA falhou:", e)

# 9.D Consolidação rápida
try:
    print("\n=== Consolidação de tabelas SHAP (top itens) ===")
    if os.path.exists("./outputs/shap/shap_rf_feature_importance.csv"):
        print(pd.read_csv("./outputs/shap/shap_rf_feature_importance.csv").head(20))
    if os.path.exists("./outputs/shap/shap_bert_token_importance.csv"):
        print(pd.read_csv("./outputs/shap/shap_bert_token_importance.csv").head(30))
    if os.path.exists("./outputs/shap/shap_llama_token_importance.csv"):
        print(pd.read_csv("./outputs/shap/shap_llama_token_importance.csv").head(30))
except Exception as e:
    print("Consolidação SHAP falhou:", e)

print("\nPipeline completo. Saídas SHAP em ./outputs/shap/ e resultados em ./results_* .")

C:\Users\chita\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


USE_CUDA=False, device=cpu

--- Pipeline para offcombr-2 ---


TypeError: 'generator' object is not subscriptable